In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import brickschema
import datetime
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.similarity_score import *
import uuid
import pickle
import time 
from rdflib import URIRef, Literal, BNode
from queries.mortar import queries as q1
%matplotlib inline

In [2]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 32}

matplotlib.rc('font', **font)

In [3]:
useful_queries = [
"""SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}""",
"""SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temperature_Setpoint .
    ?sp brick:isPointOf ?equip
}""",
"""SELECT ?sensor ?sp ?equip WHERE {
    ?sensor  rdf:type     brick:Air_Flow_Sensor .
    ?sp      rdf:type     brick:Air_Flow_Setpoint .
    ?sensor  brick:isPointOf ?equip .
    ?sp      brick:isPointOf ?equip
}""",
"""SELECT * WHERE {
    ?equip    rdf:type  brick:VAV .
    ?equip    brick:hasPoint  ?air_flow .
    ?air_flow rdf:type   brick:Supply_Air_Flow_Sensor
    }""",
"""SELECT ?vav WHERE {
    ?vav rdf:type brick:VAV
}""",
"""SELECT ?floor ?room WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?room brick:isPartOf+ ?floor .
}""",
    
"""SELECT * WHERE {
    ?oat_damper a brick:Outside_Damper .
    ?pos a brick:Damper_Position_Command .
    ?oat_damper brick:hasPoint ?pos .
    ?oat a brick:Outside_Air_Temperature_Sensor
}""",
    
"""SELECT * WHERE {
    ?equip   rdf:type  brick:VAV .
    ?equip   brick:isFedBy  ?ahu .
    ?ahu     brick:hasPoint  ?upstream_ta .
    ?equip   brick:hasPoint  ?dnstream_ta .
    ?upstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?dnstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?equip   brick:hasPoint  ?vlv .
    ?vlv     rdf:type   brick:Valve_Command
}"""
]


In [ ]:
files = os.listdir('Data - Brick models/')
brick_models = []

for file in files:
    if file.endswith(".ttl"):
        g = brickschema.Graph(load_brick=True)
        g.load_file('Data - Brick models/'+file)
        g.expand(profile="owlrl")
        brick_models.append(g)

with open("brick_models.pkl", 'wb') as fp:
    pickle.dump(brick_models, fp)


In [4]:
# files = os.listdir('Data - Brick models/')
# brick_models = []

# with open("brick_models.pkl", 'rb') as fp:
#     brick_models = pickle.load(fp)
    

In [ ]:
results = {}
output_queries = {}
unique_buildings = []
for query in useful_queries[-1:]:
    max_level = 1
    G = get_fast_relaxed_graph(query=query, max_level=max_level)
    already_done_queries = []
    original_uuid = list(G.nodes())[0]
    original_query = G.nodes()[original_uuid]['query']
    results_query = {}
    output_queries_buildings = {}
    select_statement = query.split("{")[0] + "{\n"
    
    building_idx = 0
    while building_idx < len(brick_models):
        g = brick_models[building_idx]
        scores = {}
        results_building = {}
        for idx, node in G.nodes().data():    
            relaxed_query = list(node['query'])
            level = node['level']
            
            if level > max_level: 
                continue
            else:
                brick_query = generate_brick_query_from_node(relaxed_query, select_statement=select_statement)
                res = run_brick_query(building_model=g, query=brick_query)
                
                if len(res) > 0:
                    if building_idx not in unique_buildings:
                        unique_buildings.append(building_idx)
                        break
        print("{} done, current unique buldings = {}".format(building_idx, len(unique_buildings)))
        building_idx+=1

0 done, current unique buldings = 0
1 done, current unique buldings = 0
2 done, current unique buldings = 0
3 done, current unique buldings = 1
4 done, current unique buldings = 2
5 done, current unique buldings = 3
6 done, current unique buldings = 4
